In [20]:
import sys
sys.path.append('..')
from leetcode.unionfind.uf import UF

In [21]:
# Union Find
from typing import List
from itertools import combinations

class Solution:
    def generateSentences(self, synonyms: List[List[str]], text: str) -> List[str]:
        synonyms = [set(s) for s in synonyms]
        roots = set() # store root indexes of connected components
        N = len(synonyms)
        
        uf = UF(N)
        for i, j in combinations(range(N), 2):
            if synonyms[i] & synonyms[j]: # has overlap
                uf.union(i, j) # union two synonym indexes
                
        # for each root of connected component, union all words into one big set
        for i in range(N):
            root = uf.find(i)
            roots.add(root)
            synonyms[root] |= synonyms[i]
            
        def dfs(i: int, path: List[str], res: List[List[str]]):
            if i == W:
                res.append(path)
                return
            if founds[i] == -1:
                dfs(i+1, path + [words[i]], res)
            else:
                for s in synonyms[founds[i]]:
                    dfs(i+1, path + [s], res)
                    
        words = text.split()
        W = len(words)
        founds = [-1 for i in range(W)] # check each word's existence in synonyms
        for i, w in enumerate(words):
            for root in roots:
                if w in synonyms[root]:
                    founds[i] = root
                    break
        res = []
        dfs(0, [], res)
        return sorted([' '.join(lst) for lst in res])

In [27]:
# short version of previous Solution using itertools.product(*args)
from typing import List
from itertools import *

class Solution:
    def generateSentences(self, synonyms: List[List[str]], text: str) -> List[str]:
        synonyms = [set(s) for s in synonyms]
        roots = set() # store root indexes of connected components
        N = len(synonyms)
        
        uf = UF(N)
        for i, j in combinations(range(N), 2):
            if synonyms[i] & synonyms[j]: # has overlap
                uf.union(i, j) # union two synonym indexes
                
        # for each root of connected component, union all words into one big set
        for i in range(N):
            root = uf.find(i)
            roots.add(root)
            synonyms[root] |= synonyms[i]
            
        words = text.split()
        W = len(words)
        C = [] # store all possible words in each index
        for i, w in enumerate(words):
            found = False
            for root in roots:
                if w in synonyms[root]:
                    C.append(sorted(list(synonyms[root])))
                    found = True
                    break
            if not found:
                C.append([w])
        # Cartesian product: https://docs.python.org/3/library/itertools.html?#itertools.product
        # e.g. itertools.product([['I am'],['cheerful','happy','joy'],['today but was'],['sad','sorrow'],['yesterday']])
        return [' '.join(candidate) for candidate in product(*C)] # unpack array C, then for each column, it's a possible word set

In [41]:
# DFS
# @awice
from typing import List
from collections import defaultdict
from itertools import *

class Solution:
    def generateSentences(self, synonyms: List[List[str]], text: str) -> List[str]:
        def dfs(curword: str):
            for v in graph[curword]:
                if v not in seen:
                    seen.add(v)
                    dfs(v)
                
        graph = defaultdict(list)
        for s in synonyms:
            u, v = s[0], s[1]
            graph[u].append(v)
            graph[v].append(u)
#         print(graph)
        
        words = text.split()
        C = []
        for w in words:
            if w not in graph:
                C.append([w])
            else:
                seen = {w}
                dfs(w)
                C.append(sorted(list(seen)))
#             print(C)
        return [' '.join(candidate) for candidate in product(*C)]

In [42]:
Solution().generateSentences(synonyms = [["happy","joy"],["sad","sorrow"],["joy","cheerful"]], text = "I am happy today but was sad yesterday")

['I am cheerful today but was sad yesterday',
 'I am cheerful today but was sorrow yesterday',
 'I am happy today but was sad yesterday',
 'I am happy today but was sorrow yesterday',
 'I am joy today but was sad yesterday',
 'I am joy today but was sorrow yesterday']